In [ ]:
def fetch_with_pagination(max_pages=5):
    all_data = []
    url = 'https://api.coingecko.com/api/v3/coins/markets'
    
    for page in range(1, max_pages + 1):
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': 50,
            'page': page,
            'sparkline': 'false'
        }
        
        response = requests.get(url, params=params)
        
        if response.status_code != 200:
            break
            
        data = response.json()
        
        if not data:
            break
            
        all_data.extend(data)
        time.sleep(1.5)
    
    return all_data

In [ ]:
def parse_nested_json(data, key_path):
    keys = key_path.split('.')
    result = data
    
    for key in keys:
        if isinstance(result, dict):
            result = result.get(key)
        elif isinstance(result, list) and key.isdigit():
            result = result[int(key)]
        else:
            return None
            
    return result

In [ ]:
sample_csv_data = {
    'crypto_id': df['id'].head(50).values,
    'symbol': df['symbol'].head(50).values,
    'additional_metric': np.random.uniform(0.5, 2.0, 50)
}
df_csv = pd.DataFrame(sample_csv_data)
df_csv.to_csv('crypto_additional.csv', index=False)
df_csv.head()

In [ ]:
df.info()

In [ ]:
missing_before = df.isnull().sum()
print(missing_before[missing_before > 0])

In [ ]:
string_cols = ['id', 'symbol', 'name']
for col in string_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip().str.lower()

In [ ]:
outlier_cols = ['current_price', 'market_cap', 'total_volume']

for col in outlier_cols:
    if col in df.columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
        print(f"{col}: {outliers} outliers detected")

In [ ]:
def categorize_market_cap(cap):
    if cap > 10_000_000_000:
        return 'large'
    elif cap > 1_000_000_000:
        return 'medium'
    else:
        return 'small'

df['market_cap_category'] = df['market_cap'].apply(categorize_market_cap)

In [ ]:
df['has_max_supply'] = df['max_supply'].notna().astype(int)

In [ ]:
category_stats = df.groupby('market_cap_category').agg({
    'current_price': ['mean', 'median', 'std'],
    'market_cap': ['sum', 'mean'],
    'total_volume': ['sum', 'mean'],
    'price_change_percentage_24h': ['mean', 'std'],
    'id': 'count'
})
category_stats

In [ ]:
correlation_cols = ['current_price', 'market_cap', 'total_volume', 'price_change_percentage_24h', 
                    'volatility_24h', 'volume_to_market_cap']
correlation_matrix = df[correlation_cols].corr()
correlation_matrix

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.dpi'] = 100

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8}, 
            fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Matrix - Crypto Metrics', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

category_market_cap = df.groupby('market_cap_category')['market_cap'].sum()
axes[0].pie(category_market_cap, labels=category_market_cap.index, autopct='%1.1f%%',
            startangle=90, colors=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0].set_title('Total Market Cap Distribution by Category', fontsize=14, fontweight='bold')

trend_counts = df['price_trend'].value_counts()
trend_colors = {'strong_gain': '#00CC66', 'gain': '#66FF99', 
                'loss': '#FF9999', 'strong_loss': '#CC0000'}
colors = [trend_colors.get(x, 'gray') for x in trend_counts.index]
axes[1].pie(trend_counts, labels=trend_counts.index, autopct='%1.1f%%',
            startangle=90, colors=colors)
axes[1].set_title('Price Trend Distribution', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
top_20 = df.nlargest(20, 'market_cap')

fig, ax = plt.subplots(figsize=(14, 8))

colors = ['green' if x > 0 else 'red' for x in top_20['price_change_percentage_24h']]
bars = ax.bar(top_20['symbol'], top_20['current_price'], color=colors, alpha=0.7, edgecolor='black')

ax.set_title('Top 20 Cryptocurrencies - Current Price', fontsize=16, fontweight='bold')
ax.set_xlabel('Symbol', fontsize=12)
ax.set_ylabel('Price (USD)', fontsize=12)
ax.tick_params(axis='x', rotation=45)
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
pivot_data = df.pivot_table(
    values='current_price',
    index='market_cap_category',
    columns='price_trend',
    aggfunc='mean',
    fill_value=0
)

plt.figure(figsize=(10, 6))
sns.heatmap(pivot_data, annot=True, fmt='.2f', cmap='YlOrRd', 
            linewidths=0.5, cbar_kws={'label': 'Avg Price (USD)'})
plt.title('Average Price: Market Cap Category vs Price Trend', 
          fontsize=14, fontweight='bold', pad=20)
plt.xlabel('Price Trend', fontsize=12)
plt.ylabel('Market Cap Category', fontsize=12)
plt.tight_layout()
plt.show()